Модель способна выполнять:


1.   Линейную классификацию
2.   Нелинейную классификацию
3. Регрессию
4. Выявление выбросов

Методы SVM особенно хорошо подходят для классификации сложных, но небольших или средних наборов данных.




# Линейная классификация SVM

### Классификация с широким зазором

Граница решений классификатора SVM представляет собой линию, находящиеюся **максимально возможно далеко** (как можно более широкой) от ближайших обучающих образцов. Такие образцы называются – опорные вектора

Алгоритм чувствителен к масштабам признаков. Поэтому в предорбработке нужно использовать `StandartScaler`

## Классификация с жестким и мягким зазором

**Жесткий зазор** определяет что все образцы находятся вне полосы.

**Мягкий зазор** позволяет нарушать границу, но с ограничением количества нарушений.

Проблемы жесткого зазора:

1. Невозможность разлеить линейно несепарабельные наборы (если присутствуют выбросы)
2. Если наборы сепарабельны, но присутствует один выброс, который близок к "другому классу" экземпляров, то такая модель не будет хорошо обобщаться

В классах SVM библиотеки sklearn возможно управлять балансом "ширина полосы/ количество нарушений" гиперпараметром "C"

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

In [ ]:
iris = datasets.load_iris()

In [ ]:
X = iris["data"][:, (2, 3)] # длина и ширина лепестка
y = (iris["target"] == 2).astype(np.float64) # ирис вингриский

In [ ]:
svm_clf = Pipeline([
                ("scaler", StandardScaler()),
                ("linear_svc", LinearSVC(C=1, loss="hinge"))])

In [ ]:
svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, loss='hinge'))])

In [ ]:
svm_clf.predict([[5.5, 1.7]])

array([1.])

SVM не выдает выероятности для каждого класса

# Нелинейная классификация SVM

Если наборы данных далеки от линейно сепарабельных, то пользуются **нелинейной классификацияей**

Как обрабатывать нелинейные нвборы?

1. Добавить дополнительные полиноминальные признаки

## Добавление полиноминальных признаков

In [ ]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt

polynomial_svm_clf = Pipeline([
        ("poly_features", PolynomialFeatures(degree=3)),
        ("scaler", StandardScaler()),
        ("svm_clf", LinearSVC(C=10, loss="hinge", random_state=42))
    ])

polynomial_svm_clf.fit(X, y)

Pipeline(steps=[('poly_features', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('svm_clf', LinearSVC(C=10, loss='hinge', random_state=42))])

## Полиноминальное ядро (ядерный трюк)

При низкой полиноминальной степени ПЯ не способно справляться с очень сложными наборами данных, а при высокой оно создает большое количество признаков, делая модель медленной.

In [ ]:
from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
    ])

'''полиноминальное ядро 3 степени, coef0 – насколько сильно полиномы высокой 
степени влияют на модель в сравнении с полиномами низкой'''
poly_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=1, kernel='poly'))])

In [ ]:
poly_kernel_svc_clf.fit(X, y)

## Добавление признаков близости

Вторая методика решения нелинейных задач

*опрелелим* функцию близости как "гауссову радиальную базисную функцию" – **RBF**

Простейший подход к созданию ориениров заключается в **положении ориентира по мету каждого образца**. При таком подходе создается много измерений и тем самым растут шанты на то, что трансформированный набор данных будет линейно сепарабельным. 

Но **минус** заключается в том, что обучающий набор с m образцами и n признаками трансформируется в набор с m образцами и m признаками.

m x n ---> m x m

In [ ]:
rbf_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
    ])
rbf_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=0.001, gamma=5))])

Класс SVC:

1. kernel – ядро
2. gamma – увеличение --> сужение функции (сфера влияния каждого образца уменьшается)
3. C – увеличение --> сужение зазора

## Гауссово ядро RBF

# Регрессия SVM

LinearSVR

Прием заключается в инвертировании цели

In [ ]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5, random_state=42)
svm_reg.fit(X, y)

LinearSVR(epsilon=1.5, random_state=42)

In [ ]:
from sklearn.svm import SVR

svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1, gamma="auto")
svm_poly_reg.fit(X, y)

SVR(C=100, degree=2, gamma='auto', kernel='poly')

# Классы из sklearn для SVM

### Классификация

**LinearSVC** – для линейных методов SVM

*   loss="hinge" – желатоельно
*   dual=False
*   C=10

Нужно центрировать `StandardScaler`

Можно добавить полиноминалные признаки классом `PolynominalFeatures`

* degree=3 (example)

1. PolynominalFeatures (не обязательно)
2. StandardScaler
3. LinearSVC

Добавление малого количества полиноминальных признаков не сможет справиться с очень сложными наборами, а при высокой создает огромное количество признаков. Иногда применяют ядерный трюк, который поддерживается классом `SVC`

---


**SVC** – поддерживает ядерный трюк. Может использоваться для линейной классификации

* kernel = "linear","poly","rbf"
  * gamma(rbf) = 5
* degree=3
* coef0=1
* C=1


---


**SGDClassifier** – на основе ГС

*   loss="hinge"
*   alpha

### Регрессия

**LinearSVR** –
* epsilon = 1.5


---


**SVR** – поддерживает ядерный трюк
* kernel="poly"
* degree=2
* C=100
* epsilon=0.1
* gamma = "auto"